In [18]:
# %matplotlib inline
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import pandas as pd
from pandas_profiling import ProfileReport
from scipy.stats import pearsonr
from scipy import stats
from statsmodels.stats.anova import AnovaRM
from sklearn.covariance import GraphicalLasso, graphical_lasso,EmpiricalCovariance
from sklearn.decomposition import PCA, FastICA
from nilearn.maskers import NiftiMasker
from nilearn.maskers import NiftiLabelsMasker
from nilearn.maskers import NiftiMapsMasker
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from nilearn.plotting import plot_stat_map, show, plot_epi, plot_prob_atlas, find_xyz_cut_coords, plot_roi
from nilearn import image
from nilearn.plotting import plot_stat_map, show
from nilearn import plotting
import seaborn as sns
import os
import importlib

import gi_gmn
import gi_gmn.utils as giu
import gi_gmn.models as gim

In [13]:
NP_train = 30
NC_train = 25
NP = 42
NC = 36
NPRE = 16
NP_test = NP - NP_train
NC_test = NC - NC_train
TR = 170
NRecord = 2
ROI = 120

all_col = ['Precentral_L', 'Precentral_R', 'Frontal_Sup_2_L', 'Frontal_Sup_2_R', 'Frontal_Mid_2_L', 'Frontal_Mid_2_R', 'Frontal_Inf_Oper_L', 'Frontal_Inf_Oper_R', 'Frontal_Inf_Tri_L', 'Frontal_Inf_Tri_R', 'Frontal_Inf_Orb_2_L', 'Frontal_Inf_Orb_2_R', 'Rolandic_Oper_L', 'Rolandic_Oper_R', 'Supp_Motor_Area_L', 'Supp_Motor_Area_R', 'Olfactory_L', 'Olfactory_R', 'Frontal_Sup_Medial_L', 'Frontal_Sup_Medial_R', 'Frontal_Med_Orb_L', 'Frontal_Med_Orb_R', 'Rectus_L', 'Rectus_R', 'OFCmed_L', 'OFCmed_R', 'OFCant_L', 'OFCant_R', 'OFCpost_L', 'OFCpost_R', 'OFClat_L', 'OFClat_R', 'Insula_L', 'Insula_R', 'Cingulate_Ant_L', 'Cingulate_Ant_R', 'Cingulate_Mid_L', 'Cingulate_Mid_R', 'Cingulate_Post_L', 'Cingulate_Post_R', 'Hippocampus_L', 'Hippocampus_R', 'ParaHippocampal_L', 'ParaHippocampal_R', 'Amygdala_L', 'Amygdala_R', 'Calcarine_L', 'Calcarine_R', 'Cuneus_L', 'Cuneus_R', 'Lingual_L', 'Lingual_R', 'Occipital_Sup_L', 'Occipital_Sup_R', 'Occipital_Mid_L', 'Occipital_Mid_R', 'Occipital_Inf_L', 'Occipital_Inf_R', 'Fusiform_L', 'Fusiform_R', 'Postcentral_L', 'Postcentral_R', 'Parietal_Sup_L', 'Parietal_Sup_R', 'Parietal_Inf_L', 'Parietal_Inf_R', 'SupraMarginal_L', 'SupraMarginal_R', 'Angular_L', 'Angular_R', 'Precuneus_L', 'Precuneus_R', 'Paracentral_Lobule_L', 'Paracentral_Lobule_R', 'Caudate_L', 'Caudate_R', 'Putamen_L', 'Putamen_R', 'Pallidum_L', 'Pallidum_R', 'Thalamus_L', 'Thalamus_R', 'Heschl_L', 'Heschl_R', 'Temporal_Sup_L', 'Temporal_Sup_R', 'Temporal_Pole_Sup_L', 'Temporal_Pole_Sup_R', 'Temporal_Mid_L', 'Temporal_Mid_R', 'Temporal_Pole_Mid_L', 'Temporal_Pole_Mid_R', 'Temporal_Inf_L', 'Temporal_Inf_R', 'Cerebelum_Crus1_L', 'Cerebelum_Crus1_R', 'Cerebelum_Crus2_L', 'Cerebelum_Crus2_R', 'Cerebelum_3_L', 'Cerebelum_3_R', 'Cerebelum_4_5_L', 'Cerebelum_4_5_R', 'Cerebelum_6_L', 'Cerebelum_6_R', 'Cerebelum_7b_L', 'Cerebelum_7b_R', 'Cerebelum_8_L', 'Cerebelum_8_R', 'Cerebelum_9_L', 'Cerebelum_9_R', 'Cerebelum_10_L', 'Cerebelum_10_R', 'Vermis_1_2', 'Vermis_3', 'Vermis_4_5', 'Vermis_6', 'Vermis_7', 'Vermis_8', 'Vermis_9', 'Vermis_10']


In [14]:
!pwd


/users/grad/fmoradi/Graphical_LASSO/package/tests


In [15]:
#reading data loading thm into matrix size of N*TR * ROI
control_data  = pd.read_csv('../../Ataxia/control_data_twoyear.csv',index_col=0).to_numpy()
patient_data = pd.read_csv('../../Ataxia/patient_data_twoyear.csv',index_col=0).to_numpy()
patient_data.shape, control_data.shape


((14280, 120), (12240, 120))

In [16]:
importlib.reload(gi_gmn)
importlib.reload(gi_gmn.utils)
# group level GMN - performance on test and saving preciison matrix 
runs = 5

# best model for year one and year two on cross sectional
alpha_p = 0.07
alpha_c = 0.07

a_p , a_c = [],[]



for i in range(runs):

    #randomly select controls for test
    index_control = [np.random.choice(np.arange(0 , NC - 1), size = NC_test,replace = False)]
    control = [j for j in range(NC)]
    index_control_test = list(index_control[0])
    index_control_train = list(set(control)  - set(index_control_test))

    #randomly select patients for test
    index_patient = [np.random.choice(np.arange(0 , NP - 1), size = NP_test,replace = False)]
    patient = [j for j in range(NP)]
    index_patient_test = list(index_patient[0])
    index_patient_train = list(set(patient)  - set(index_patient_test))


    parts_c = giu.split_into_chunks(control_data, TR*NRecord)
    parts_p = giu.split_into_chunks(patient_data, TR*NRecord)


    control_test_data = np.array([part for i,part in enumerate(parts_c) if i in index_control_test ]).reshape(NC_test*TR*NRecord,ROI)
    control_train_data = np.array([part for i,part in enumerate(parts_c) if i in index_control_train ]).reshape(NC_train*TR*NRecord,ROI)

    patient_test_data = np.array([part for i,part in enumerate(parts_p) if i in index_patient_test ]).reshape(NP_test*TR*NRecord,ROI)
    patient_train_data = np.array([part for i,part in enumerate(parts_p) if i in index_patient_train ]).reshape(NP_train*TR*NRecord,ROI)

    

    # get the distribution and Convatraince matrix of the model
    patientGL = GraphicalLasso(alpha = alpha_p,max_iter = 350)
    controlGL = GraphicalLasso(alpha = alpha_c,max_iter = 360)


    # train the model
    patient_model = patientGL.fit(patient_train_data)
    control_model = controlGL.fit(control_train_data)


    cov_patient = np.around(patient_model.precision_, decimals=3)
    cov_control = np.around(control_model.precision_, decimals=3)

    # print(degree_sparsity(cov_control),degree_sparsity(cov_patient))

    #save the precision matrix - group level
    # cov_control = pd.DataFrame(cov_control)
    # cov_control.to_csv('../../Ataxia/control_precision.csv',index = False)
    # cov_patient = pd.DataFrame(cov_patient)
    # cov_patient.to_csv('../../Ataxia/patient_precision.csv',index = False)

    #test
    recall_p = 0 
    recall_c = 0

    for indx in range(0,patient_test_data.shape[0],TR*NRecord):
        log_prob_control = controlGL.score(patient_test_data[indx:indx+TR*NRecord,:])
        log_prob_patient = patientGL.score(patient_test_data[indx:indx+TR*NRecord,:])
        # print(log_prob_patient-log_prob_control)
        if (log_prob_control<log_prob_patient):
            recall_p = recall_p + 1
            
    for indx in range(0,control_test_data.shape[0],TR*NRecord):
        log_prob_control = controlGL.score(control_test_data[indx:indx+TR*NRecord,:])
        log_prob_patient = patientGL.score(control_test_data[indx:indx+TR*NRecord,:])
        # print(log_prob_patient-log_prob_control)
        if (log_prob_control>log_prob_patient):
            recall_c = recall_c + 1


    recall_p = recall_p / NP_test
    recall_c = recall_c / NC_test
    a_p.append(recall_p)
    a_c.append(recall_c)

print(np.array(a_p).mean(),np.array(a_p).std(), np.array(a_c).mean(), np.array(a_c).std())

DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
DEBUG
0.5166666666666666 0.17795130420052185 0.9636363636363636 0.04453617714151235


In [26]:
importlib.reload(gi_gmn)
importlib.reload(gi_gmn.models)

model = gim.GroupSubjectGMN()
model.fit()

Model: Group-Subject GMN
Model fitted successfully!


In [1]:
import gi_gmn.visualization as vis
